## [Primer pipeline con RAG](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)

### 1. Obtención e indexación de documentos

In [1]:
#! Solo es para haystack sepa que tutorial se esta ejecutando
from haystack.telemetry import tutorial_running
tutorial_running(27)

In [2]:
# Inicializar el DocumentStore
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

In [3]:
# Obtener los datos de ejemplo
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

In [4]:
# Inicializar Embedder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model='sentence-transformers/all-MiniLM-L6-v2')

# Descargar el modelo
doc_embedder.warm_up()

In [5]:
# Indexar los documentos
docs_with_embeddings = doc_embedder.run(docs)
#docs_with_embeddings

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
# Escribir documentos en el DocumentStore
document_store.write_documents(docs_with_embeddings["documents"])

# Mostrar los documentos indexados
# document_store.filter_documents()

151

### 2. Construyendo el RAG

In [7]:
# Inicializar el Text Embedder, se debe usar el mimo modelo con el que se crearon los embeddings
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model='sentence-transformers/all-MiniLM-L6-v2')

In [8]:
# Iniciar el recuperador -> este retriver obtendrá los documentos relevantes para una consulta
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriver = InMemoryEmbeddingRetriever(document_store)

In [9]:
# Definir una platilla para el prompt
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=template)

In [10]:
prompt_builder

Inputs:
  - documents: Any
  - question: Any
  - template: Optional[str]
  - template_variables: Optional[Dict[str, Any]]
Outputs:
  - prompt: str

In [11]:
# Inicializar un generador
from haystack.utils import Secret
from haystack.components.generators import OpenAIGenerator

generator = OpenAIGenerator(
                            api_key=Secret.from_env_var("OPENAI_API_KEY"),
                            model="gpt-4o-mini-2024-07-18"
                            )

In [12]:
# Construir el pipeline
from haystack import Pipeline

basic_rag_pipeline = Pipeline()

# Agregar los componentes al pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriver)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Conectar los componentes entre sí
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [17]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Colossus of Rhodes was a statue of the Greek sun-god Helios, standing approximately 70 cubits, or about 33 meters (108 feet) tall. According to ancient accounts, it was constructed with iron tie bars covered by brass plates that formed the skin of the statue. The interior of the statue was filled with stone blocks as construction progressed.

While scholars lack detailed accounts of its full appearance, they have a general idea based on contemporary depictions of Helios. The head of the statue was likely adorned with curly hair and had evenly spaced spikes, which represented rays of sunlight. These features are similar to images found on coins from Rhodes dating back to that period.

The Colossus was said to stand on a 15-meter-high (49-foot) white marble pedestal at the entrance of Rhodes' harbor. The statue itself was designed in a manner that conveyed a sense of grandeur and power, celebrating the Rhodians' victory and independence after defending their city against an attack. A

In [ ]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]